In [1]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol with an attacker, to demonstrate that the attacker can be detected.

# for debugging
def show_latex(circuit, bits):
    state = Statevector.from_int(0, 2**bits)
    state = state.evolve(circuit)
    display(state.draw("latex"))

def randomProb(render=False, ZeroStat=1/2, phi=math.pi/2):
    # note: this is is radians.
    theta = 2 * math.acos(math.sqrt(ZeroStat) )    
    
    circuit = QuantumCircuit(1)
    circuit.r(theta, phi, 0)
    
    # small interrupt to render latex for our state
    if (render):
        state = Statevector.from_int(0, 2**1)
        state = state.evolve(circuit)
        display(state.draw("latex"))
    
    circuit.measure_all()
    
    backend = BasicSimulator()
    compiled = transpile(circuit, backend)
    job_sim = backend.run(compiled, shots=1)
    results_sim = job_sim.result()
    counts = results_sim.get_counts()
    return int(list(counts.keys())[0])

def random50(render=False):
    return randomProb(render, 1/2, math.pi/2)

def random33(render=False):
    return randomProb(render, 1/3, phi=math.pi/2)


print("random 50/50 state:")
random50(True)
print("random 33/66 state:")
random33(True)


random 50/50 state:


<IPython.core.display.Latex object>

random 33/66 state:


<IPython.core.display.Latex object>

0

In [3]:
############################################# CONSTANTS #############################################

root2 = math.sqrt(2)

denom1 = math.sqrt(4 + 2*root2)
denom2 = math.sqrt(4 - 2*root2) 


W = [ [ -1 / denom1 , (1 + root2) / denom1 ],
      [  1 / denom2 , (root2 - 1) / denom2 ] ]

V = [ [  1 / denom1 , (1 + root2) / denom1 ],
      [ -1 / denom2 , (root2 - 1) / denom2 ] ]

############################################# CODE ###################################################


targetKeyLen = 100

finalKey = []
S = {(1,1):[0, 0], (1,3):[0, 0], (3,1):[0, 0], (3,3):[0, 0]}

for qbitI in range(int(targetKeyLen * 9/2)):
    # first, create the entangled pair by building the bell state. (with an extra qbit in the circuit for candice)
    circuit = QuantumCircuit(3)
    circuit.x(1)
    circuit.h(1)
    circuit.cx(1, 2)
    circuit.x(1)

    # candice copies bob's qbit
    circuit.cx(1, 0)

    # show_latex(circuit, 2)
    # display(circuit.draw("mpl"))

    # Alice chooses an operator Ai randomly from her set of three
    if random33() == 0:
        i = 1
        circuit.h(2)
    elif random50() == 0:
        i = 2        
        circuit.unitary(W, [2])
    else:
        i = 3
        # do nothing

    # Bob chooses an operator Bj randomly from his set of three
    if random33() == 0:
        j = 1
        circuit.unitary(W, [1])
    elif random50() == 0:
        j = 2
        # do nothing
    else:
        j = 3
        circuit.unitary(V, [1])

    # Candice pretends to be bob by choosing an operator Bj randomly from her set of three
    if random33() == 0:
        k = 1
        circuit.unitary(W, [0])
    elif random50() == 0:
        k = 2
        # do nothing
    else:
        k = 3
        circuit.unitary(V, [0])
    
    # bob agreed to invert his bits
    circuit.x(1)
    # Candice does the same
    circuit.x(0)

    # alice, bob, and candice all measure their qbits.
    circuit.measure_all()
    backend = BasicSimulator()
    compiled = transpile(circuit, backend)
    job_sim = backend.run(compiled, shots=1)
    results_sim = job_sim.result()
    counts = results_sim.get_counts()
    measurements = list(counts.keys())

    # operator choices are shared now.

    if (i == 3 and j == 2) or (i == 2 and j == 1): # these combinations are used for measurement
        finalKey.append(measurements[0])
    elif (i == 1 and j == 2) or (i == 2 and j == 2) or (i == 2 and j == 3): # these combinations are discarded
        pass
    else: # these remaining basis choices are combinations usable in the bell state entanglement test
        key = (i, j)
        measurements = [int(val) * -2 + 1 for val in measurements[0]] # convert {0, 1} -> {-1, 1}
        newVal = S[key][0] + measurements[0] * measurements[1] # multiply and add to the total sum
        newFreq = S[key][1] + 1 # track the frequency
        
        S[i, j] = [newVal, newFreq] # update the record for this particular state choice combination
    
    # display(circuit.draw("mpl"))

# if a state combination appeared 0 times.. change it's count to 1 to avoid division error
for key in S:
    if S[key] == [0, 0]:
        S[key] = [0, 1]

print(f"key ratio: {int(targetKeyLen * 9/2)/len(finalKey)}, ideally {9/2=}")

print()
print(f"final key (len {len(finalKey)}):")
print(" ".join(finalKey))
print()
print("random choices distribution like (i, j): [sum, occurances]")
print(S)

print()
print("S components:")
print(f"{S[(1, 1)][0]} / {S[(1, 1)][1]} = {S[(1, 1)][0] / S[(1, 1)][1]}")
print(f"{S[(1, 3)][0]} / {S[(1, 3)][1]} = {S[(1, 3)][0] / S[(1, 3)][1]}")
print(f"{S[(3, 1)][0]} / {S[(3, 1)][1]} = {S[(3, 1)][0] / S[(3, 1)][1]}")
print(f"{S[(3, 3)][0]} / {S[(3, 3)][1]} = {S[(3, 3)][0] / S[(3, 3)][1]}")

print()
print("S sum:")
Ssum = abs((S[(1, 1)][0] / S[(1, 1)][1]) - # X, W
            (S[(1, 3)][0] / S[(1, 3)][1]) + # X, V
            (S[(3, 1)][0] / S[(3, 1)][1]) + # Z, W
            (S[(3, 3)][0] / S[(3, 3)][1])) # Z, V

print()
print(f"{Ssum=}, {2 * 2**0.5=}")

key ratio: 4.545454545454546, ideally 9/2=4.5

final key (len 99):
001 110 001 111 111 001 111 001 000 101 000 000 000 000 000 110 111 001 000 110 110 110 000 000 011 000 111 100 110 110 101 110 111 010 001 110 111 000 000 000 000 111 111 000 111 000 111 001 111 111 110 110 000 111 100 000 010 100 110 111 111 000 100 000 001 000 110 111 110 000 111 111 001 111 001 110 110 110 110 111 001 001 101 001 000 000 111 110 111 000 110 010 101 110 001 111 100 111 111

random choices distribution like (i, j): [sum, occurances]
{(1, 1): [2, 54], (1, 3): [6, 52], (3, 1): [-40, 58], (3, 3): [-39, 45]}

S components:
2 / 54 = 0.037037037037037035
6 / 52 = 0.11538461538461539
-40 / 58 = -0.6896551724137931
-39 / 45 = -0.8666666666666667

S sum:

Ssum=1.634669417428038, 2 * 2**0.5=2.8284271247461903
